In [ ]:
!git clone https://github.com/yixuantt/FinEntity.git
%cd FinEntity

Cloning into 'FinEntity'...
remote: Enumerating objects: 118, done.
remote: Total 118 (delta 0), reused 0 (delta 0), pack-reused 118 (from 1)
Receiving objects: 100% (118/118), 28.24 MiB | 21.39 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Error downloading object: model_bert_crf (24132d8): Smudge error: Error downloading model_bert_crf (24132d85e1ad90e91d9688e5ddaf6f520396644386817fa2f877f21a58ea84fe): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/FinEntity/.git/lfs/logs/20250505T033102.782069901.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: model_bert_crf: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

/content/FinEntity


In [ ]:
!pip install transformers datasets seqeval scikit-learn accelerate pytorch_crf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:0

In [ ]:
# load model
import pickle

with open('model_bert_crf', 'rb') as f:
    model = pickle.load(f)



In [ ]:
import json
import pandas as pd
from sequence_aligner.dataset import PredictDatasetCRF
from sequence_aligner.containers import PredictBatch
from transformers import BertTokenizerFast
from sequence_aligner.labelset import LabelSet
from util.train import predict4news
from torch import cuda
import torch
import warnings
warnings.filterwarnings('ignore')

# 1) Load articles
df = pd.read_csv("articles_filtered.csv", parse_dates=["Date"])
df = df.dropna(subset=["text"])

# 2) Prepare tokenizer, label set, device, model
tokenizer = BertTokenizerFast.from_pretrained("yiyanghkust/finbert-pretrain")
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"])
device = "cuda:0" if cuda.is_available() else "cpu"
model.to(device)
model.eval()

df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.dropna(subset=["Date"])

# 3) Build items for prediction
items = []
for _, row in df.iterrows():
    items.append({
        "date_publish": row["Date"].strftime("%Y-%m-%d"),
        "content": row["text"]
    })

# 4) Run CRF inference with progress tracking
save_list = []
total = len(items)

for i, item in enumerate(items):
    predict4news(model, item, device, tokenizer, label_set, save_list)

    if (i + 1) % max(1, total // 20) == 0 or (i + 1) == total:
        print(f"Progress: {100 * (i + 1) / total:.1f}%")

# 5) Save predictions
with open("entitylevel_predictions.json", "w", encoding="utf-8") as f:
    json.dump(save_list, f, ensure_ascii=False, indent=2)

print(f"Done!")


Progress: 5.0%
Progress: 10.0%
Progress: 15.0%
Progress: 20.0%
Progress: 25.0%
Progress: 29.9%
Progress: 34.9%
Progress: 39.9%
Progress: 44.9%
Progress: 49.9%
Progress: 54.9%
Progress: 59.9%
Progress: 64.9%
Progress: 69.9%
Progress: 74.9%
Progress: 79.8%
Progress: 84.8%
Progress: 89.8%
Progress: 94.8%
Progress: 99.8%
Progress: 100.0%
Done!
